# Certificado Digital

#### Alunos: Gabriel Schneider e Luciane Tedesco 

#### Questão 1

Crie um programa que abra a keystore e exiba as seguintes propriedades dos certificados importados:

- Quem é o proprietário do certificado ?
- Quem é o emissor do certificado?
- Identifique se o certificado é auto-assinado ou não.

%path C:\Program Files (x86)\Java\jdk1.8.0_161\bin

%keytool -genkey -v -keystore my-release-key.keystore -alias alias_name -keyalg RSA -keysize 2048 -validity 10000

In [25]:
import sys
import chilkat

# The Chilkat API can be unlocked for a fully-functional 30-day trial by passing any
# string to the UnlockBundle method.  A program can unlock once at the start. Once unlocked,
# all subsequently instantiated objects are created in the unlocked state. 
# 
# After licensing Chilkat, replace the "Anything for 30-day trial" with the purchased unlock code.
# To verify the purchased unlock code was recognized, examine the contents of the LastErrorText
# property after unlocking.  For example:
glob = chilkat.CkGlobal()
success = glob.UnlockBundle("Anything for 30-day trial")
if (success != True):
    print(glob.lastErrorText())
    sys.exit()

status = glob.get_UnlockStatus()
if (status == 2):
    print("Unlocked using purchased unlock code.")
else:
    print("Unlocked in trial mode.")

# The LastErrorText can be examined in the success case to see if it was unlocked in
# trial more, or with a purchased unlock code.
print(glob.lastErrorText())

Unlocked in trial mode.
ChilkatLog:
  UnlockBundle:
    DllDate: Oct 28 2019
    ChilkatVersion: 9.5.0.80
    UnlockPrefix: Anything for 30-day trial
    Architecture: Little Endian; 64-bit
    Language: Windows Python 3.*
    VerboseLogging: 0
    component: Global
    Component successfully unlocked using trial key
    Success.
  --UnlockBundle
--ChilkatLog



In [30]:
import sys
import chilkat

jks = chilkat.CkJavaKeyStore()
success = jks.LoadFile("112233","keyStore.jks")
if (success != True):
    print(jks.lastErrorText())
    #sys.exit()
    
#  Find out how many of each type of entry:
numTrustedCerts = jks.get_NumTrustedCerts()
numPrivateKeys = jks.get_NumPrivateKeys()

#  For each trusted certificate, access it by getting
#  it as a cert object.  Also get the alias associated with the certificate.
print("Trusted Certs:")
i = 0
pks = []
while i < numTrustedCerts :
    # cert is a CkCert
    cert = jks.GetTrustedCert(i)
    print(jks.getTrustedCertAlias(i) + ": " + cert.subjectDN())
    pks.append(cert.ExportPublicKey())
    i = i + 1

print("Private Keys:")
i = 0
while i < numPrivateKeys :
    # privKey is a CkPrivateKey
    privKey = jks.GetPrivateKey("112233",i)
    print(jks.getPrivateKeyAlias(i))
    # certChain is a CkCertChain
    certChain = jks.GetCertChain(i)

    #  The 1st certificate in the chain is the one associated with the private key.
    # cert is a CkCert
    cert = certChain.GetCert(0)
    print(cert.subjectDN())

    i = i + 1

ChilkatLog:
  LoadFile:
    DllDate: Oct 28 2019
    ChilkatVersion: 9.5.0.80
    UnlockPrefix: Anything for 30-day trial
    Architecture: Little Endian; 64-bit
    Language: Windows Python 3.*
    VerboseLogging: 0
    path: keyStore.jks
    loadJksBinary:
      magicNumber: 0xfeedfeed
      JksVersion: 2
      numEntries: 4
      alias: *.furb.br (icpedu)
      alias: gilvan justino
      alias: www.angeloni.com.br (go daddy secure certificate authority - g2)
      alias: www.guiadoestudante.abril.com.br (let's encrypt authority x3)
      keyed digest verification failed.
      keyedDigestVerified: 0
    --loadJksBinary
    Failed.
  --LoadFile
--ChilkatLog

Trusted Certs:
*.furb.br (icpedu): C=BR, ST=Santa Catarina, L=Blumenau, O=Fundacao Universidade Regional de Blumenau, CN=*.furb.br
gilvan justino: C=BR, ST=SC, L=Blumenau, OU=FURB, CN=Gilvan Justino
www.angeloni.com.br (go daddy secure certificate authority - g2): OID.1.3.6.1.4.1.311.60.2.1.3=BR, BusinessCategory=Private Organiz

#### Questão 2

- Cifre a imagem com o algoritmo AES utilizando a chave constituída de 16 bytes com caractere “A”. Gere a saída com valor “questao1.aes”.

- Utilize a chave pública do certificado importado para cifrar a chave do algoritmo simétrico. Armazene a chave num arquivo binário chamado “key.aes”.

Cifrando a imagem:

In [31]:
import io
from PIL import Image

def image_to_byte_array(image:Image):
    imgByteArr = io.BytesIO()
    image.save(imgByteArr, format=image.format)
    imgByteArr = imgByteArr.getvalue()
    return imgByteArr

# lendo os dados
jpgfile = Image.open("imagem/imagemparacifrar.jpg")
data = image_to_byte_array(jpgfile)

In [32]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

# criptografando os dados
aes_key = b'AAAAAAAAAAAAAAAA'
cipher = AES.new(aes_key, AES.MODE_EAX)
nonce = cipher.nonce
cipherdata, tag = cipher.encrypt_and_digest(data)

In [33]:
# escrevendo os dados
with open("questao1.aes","wb") as file:
    file.write(cipherdata)

Cifrando a chave do algoritmo simétrico:

In [34]:
# é possivel utilizar os outros certificados 
pks[0].SaveOpenSslPemFile("publicKey.pem")

True

In [35]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP

# criptografando a chave simétrica
recipient_key = RSA.import_key(open("publicKey.pem").read())
cipher_rsa = PKCS1_OAEP.new(recipient_key)

encrypted_aes_key = cipher_rsa.encrypt(aes_key)

print("Aes public key: " + str(aes_key) + "\n")
print("Aes public key encrypted: " + str(encrypted_aes_key))

Aes public key: b'AAAAAAAAAAAAAAAA'

Aes public key encrypted: b'\x1e\x95\xe3F\x1aLy\x93#\xac&[\xab\xfc\x92\xa7=\xf2\xaa\xe3\x99\xba\xf8\xdaC\x7f\x0f-\xcd\xf44\x97\x8c\xdf\xff\x9aHsKRD6]Hc\xf7=\x81\xc6\x92R4\xaaA\xf3DB\x83H\xfcO\xe4\x91\xcd9.k/\xbf\xfejugAX}\xbc\x9a\xa1G\x82\xe1\xbe\x07\x96\xcf\xa8:(\\\x06lHo\xd1&\x80\xfe\xa6\xe6\xf8l\n\x11\x03\x83+9\xfe\xc0FA\x03\xe8\x84\xe3\xacs\x81\x04\xd1+\x0e\xd15\x96+\rm\xeb\xdf<%w2\x01\x91W\xc9:\xb5K\xfd`\xddU\xd9V\x82g\x00\xc8ah\xe3(\xdf\x87\x1c\xb0H\x15\x9d\x87h\xd8f\x18\xa8\xd3\xb3\xa8\x07\xae6]g\xae\xdd\xfc\x82\xf7\xfd\x13<+\xc2\x94\x9eW\x02\x807\x98\xfc\xe9gL\xfc\xcf=b\x96\xaeJ=\xfcs\x8e\x08\\\x1b\x96\x1a`~\xb1\xf2\xafE?\xdc\x85\xf0\x99\xab\x93&9n]\x1a\xc4\x85m\x9d\xf6\x8d?\xf91\x12\xdac\xb0\xb8N-\xac=\xf0\xb91/\xa7\x15\xa8/\xad_\xc2\x98\xe7\x02)$\xff\xef\x16L\xe9\xed\x86s\xf6\x92T\xc5\xbb\xd0\xbe3\x8dI\xe3\xbc\t\xe7\x0e\xe7\x1a\xf6Ds\x16zE\xae\x0e#\xe4r\xc7\xb1\xbc2\'\xfep[:\xad>n\x13\xde\x01\xde\xdf\x9f=K\x8cqof\xcc\xc2Rj\x9az$\xcd-\xec\x

In [36]:
# escrevendo os dados
with open("key.aes","wb") as file:
    file.write(encrypted_aes_key)